# **라이브러리 로드**

In [4]:
from math import sqrt
import numpy as np
import pandas as pd

# 데이터 전처리 패키지
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 모델 패키지
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier
from deap import base, creator, tools, algorithms

# 모델 평가 패키지
from sklearn.metrics import mean_squared_error, accuracy_score, root_mean_squared_error

# 시각화 패키지
import matplotlib.pyplot as plt

# **차원 축소 - 변수 선택법**

## **1. 기본 모델**

### **1-1. 데이터 불러오기 및 전처리**

ToyotaCorolla.csv
- 중고차 판매 데이터 셋
- 가격, 연식, 주행 거리, 연료 유형, 엔진 크기, 옵션 정보 등 여러 변수 포함
- 목표 변수 : 'Price'(중고차 가격)

In [ ]:
# 데이터 불러오기
data = pd.read_csv("../data/ToyotaCorolla.csv")
data.head()

In [ ]:
# 데이터 확인
data.info()

### **1-2. 데이터 분리**

In [ ]:
# 데이터 전처리
# 'Price'를 종속 변수로 설정
X = data.drop(columns=['Price', 'Id', 'Model'])  # 독립 변수
y = data['Price']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=34)

# object변수 One-Hot-Encoding
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

# bool변수 int변환
X_train = X_train.astype({col: 'int' for col in X_train.select_dtypes(include='bool').columns})
X_test = X_test.astype({col: 'int' for col in X_test.select_dtypes(include='bool').columns})
X_train.info()

In [ ]:
# add_constant를 통해 상수항 생성
X_train = sm.add_constant(X_train)

# 모델 형성 및 결과 출력
model = sm.OLS(y_train, X_train).fit()
model.summary()


***

## **2. 전진선택법**

In [9]:
# X와 y 정의
variables = X_train.columns.tolist()  # 독립 변수들의 리스트 생성
y = y_train  # 종속 변수 설정

# 선택된 변수들을 저장할 리스트
forward_variables = []

# 전진 선택법에서 기준이 되는 p-value 설정
sl_enter = 0.05  # 변수를 추가할 때의 p-value 기준
sl_remove = 0.05  # 변수를 제거할 때의 p-value 기준

# 각 스텝별로 선택된 변수들 및 수정된 결정계수(R²) 기록
sv_per_step = []  # 각 단계에서 선택된 변수들
adj_r_squared_list = []  # 각 단계에서 계산된 수정 결정 계수
steps = []  # 단계 기록
step = 0  # 현재 단계

# 전진 선택법 (Forward Selection) 알고리즘 시작
while len(variables) > 0:  # 아직 선택되지 않은 변수가 있는 동안 반복
    remainder = list(set(variables) - set(forward_variables))  # 현재 선택되지 않은 변수들
    pval = pd.Series(index=remainder)  # 각 변수의 p-value를 저장할 시리즈 생성
    
    # 각 변수에 대해 모델을 피팅하여 p-value 계산
    for col in remainder:
        X = X_train[forward_variables + [col]]  # 현재 선택된 변수와 후보 변수를 포함한 독립 변수
        X = sm.add_constant(X)  # 상수(constant) 추가 (절편 계산을 위해 필요)
        forward_model = sm.OLS(y, X).fit(disp=0)  # OLS(Ordinary Least Squares) 회귀 모델 적합
        pval[col] = forward_model.pvalues[col]  # 해당 변수의 p-value 저장
    
    # p-value가 가장 낮은 변수 선택
    min_pval = pval.min()
    if min_pval < sl_enter:  # 기준 p-value(sl_enter)보다 작으면 변수 추가
        forward_variables.append(pval.idxmin())  # 가장 낮은 p-value의 변수 추가
        
        # 추가한 변수 중 p-value가 높은 변수를 제거하는 과정
        while len(forward_variables) > 0:  # 선택된 변수가 있는 동안 반복
            selected_X = X_train[forward_variables]  # 현재 선택된 변수로 독립 변수 생성
            selected_X = sm.add_constant(selected_X)  # 상수 추가
            selected_pval = sm.OLS(y, selected_X).fit(disp=0).pvalues[1:]  # 각 변수의 p-value 계산
            max_pval = selected_pval.max()  # 가장 높은 p-value 선택
            if max_pval >= sl_remove:  # 기준 p-value(sl_remove)보다 크면 변수 제거
                remove_variable = selected_pval.idxmax()  # 제거할 변수 선택
                forward_variables.remove(remove_variable)  # 변수 제거
            else:
                break  # 기준을 만족하면 제거 중단
        
        # 단계 업데이트 및 기록
        step += 1
        steps.append(step)  # 현재 단계 기록
        adj_r_squared = sm.OLS(y, sm.add_constant(X_train[forward_variables])).fit(disp=0).rsquared_adj  # 수정된 결정계수 계산
        adj_r_squared_list.append(adj_r_squared)  # 수정된 결정계수 기록
        sv_per_step.append(forward_variables.copy())  # 선택된 변수 리스트 복사 후 기록
    else:
        break  # p-value 기준을 만족하는 변수가 없으면 종료

#### **step별 선택된 변수와 r 값 시각화**

In [ ]:
# 그래프 설정
fig = plt.figure(figsize=(10, 10))  # 그래프 크기 설정
fig.set_facecolor('white')  # 그래프 배경색 설정

# 글꼴 크기 설정
font_size = 15

# x축 눈금 설정
plt.xticks(
    steps,  # x축 눈금 위치
    [f'step {s}\n' + '\n'.join(sv_per_step[i]) for i, s in enumerate(steps)],  # 각 단계의 변수 표시
    fontsize=1  # x축 눈금 폰트 크기 (작게 설정)
)

# 수정된 결정계수(adj_r_squared) 그래프
plt.plot(steps, adj_r_squared_list, marker='o')  # 각 단계의 결정계수 값 연결

# y축 레이블 설정
plt.ylabel('adj_r_squared', fontsize=font_size)

# 격자 표시
plt.grid(True)

# 그래프 출력
plt.show()

In [ ]:
# 전진선택법 적용 모델
forward_model = sm.OLS(y_train, sm.add_constant(pd.DataFrame(X_train[forward_variables]))).fit(disp=0)
forward_model.summary()


***

## **3. 후진 소거법**

In [12]:
def backward_regression(X, y,
                           initial_list=[], 
                           threshold_out = 0.05, # 후진선택시 고려할 기준 p-value   
                           feature_list = X_train.columns.tolist()
                           ):
    # 결과 저장용 리스트 초기화
    sv_per_step = [] # 각 스텝별로 선택된 변수들
    adj_r_squared_list = [] # 각 스텝별 수정된 결정계수
    steps = [] # 스텝
    step = 0

    # 초기 포함된 변수 리스트 설정
    included = feature_list

    while True:
        changed = False  # 변수 제거 여부를 추적하기 위한 플래그

        # 현재 포함된 변수들로 모델 피팅
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[feature_list]))).fit(disp=0)
        
        # p-value 계산
        pvalues = model.pvalues.iloc[1:] # 첫 번째 값(상수항)을 제외한 p-value
        worst_pval = pvalues.max() # p-value값이 가장 높은 변수 선택

        # p-value가 임계값을 초과하는 경우 변수 제거
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax() # 제거할 변수 이름
            included.remove(worst_feature) # 변수 제거
        
        # 단계 및 결과 업데이트
        step += 1
        steps.append(step)        
        adj_r_squared = sm.OLS(y, sm.add_constant(pd.DataFrame(X[feature_list]))).fit(disp=0).rsquared_adj
        adj_r_squared_list.append(adj_r_squared) # 수정된 결정계수 저장
        sv_per_step.append(included.copy()) # 현재 선택된 변수 저장
        
        # 더 이상 제거할 변수가 없으면 종료
        if not changed:
            break
      
    return included, step, steps, adj_r_squared_list, sv_per_step

# 함수 실행
backward_variables_function, step, steps, adj_r_squared_list, sv_per_step = backward_regression(X_train, y_train)


#### **Step별 선택된 변수와 r 값 시각화**

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.set_facecolor('white')
 
font_size = 15
plt.xticks(steps,[f'step {s}\n'+'\n'.join(sv_per_step[i]) for i,s in enumerate(steps)], fontsize=1)
plt.plot(steps, adj_r_squared_list, marker='o')
    
plt.ylabel('adj_r_squared',fontsize=font_size)
plt.grid(True)
plt.show()


In [ ]:
# 후진소거법 적용 모델
back_model = sm.OLS(y_train, sm.add_constant(pd.DataFrame(X_train[backward_variables_function]))).fit(disp=0)
back_model.summary()

***

## **4. stepwise**

In [ ]:
def stepwise_feature_selection(X_train, y_train, variables=X_train.columns.tolist() ):
    # 종속 변수 설정
    y = y_train 

    # 선택된 변수 리스트 초기화
    selected_variables = [] # 선택된 변수들

    # stepwise 시 고려할 기준 p-value 설정
    sl_enter = 0.05  # 변수를 추가할 기준 p-value
    sl_remove = 0.05  # 변수를 제거할 기준 p-value

    # 각 단계별 결과 저장용 리스트 초기화
    sv_per_step = [] # 각 스텝별로 선택된 변수들
    adjusted_r_squared = [] # 각 스텝별 수정된 결정계수
    steps = [] # 스텝
    step = 0

    while len(variables) > 0:
        # 선택되지 않은 변수들로 구성된 후보 리스트 생성
        remainder = list(set(variables) - set(selected_variables))
        pval = pd.Series(index=remainder) # 각 변수의 p-value 저장용 시리즈
        
        # 각 변수에 대해 모델 피팅 및 p-value 계산
        for col in remainder: 
            X = X_train[selected_variables + [col]]  # 현재 선택된 변수 + 후보 변수
            X = sm.add_constant(X)  # 상수항 추가
            model = sm.OLS(y, X).fit(disp=0)  # 회귀 모델 피팅
            pval[col] = model.pvalues[col]  # 후보 변수의 p-value 저장
    
        # p-value가 가장 낮은 변수 선택
        min_pval = pval.min()
        if min_pval < sl_enter:  # 기준 p-value보다 작으면 변수 선택
            selected_variables.append(pval.idxmin())
            
            # 변수 제거 과정
            while len(selected_variables) > 0:
                selected_X = X_train[selected_variables]  # 선택된 변수로 모델 피팅
                selected_X = sm.add_constant(selected_X)  # 상수항 추가
                selected_pval = sm.OLS(y, selected_X).fit(disp=0).pvalues[1:]  # 상수항 제외한 p-value 계산
                max_pval = selected_pval.max()  # 가장 높은 p-value 선택
                if max_pval >= sl_remove:  # 기준 p-value보다 크면 변수 제거
                    remove_variable = selected_pval.idxmax()  # 제거할 변수 선택
                    selected_variables.remove(remove_variable)  # 변수 제거
                else:
                    break
            
            # 단계별 결과 업데이트
            step += 1
            steps.append(step)  # 현재 단계 기록
            adj_r_squared = sm.OLS(y, sm.add_constant(X_train[selected_variables])).fit(disp=0).rsquared_adj  # 수정된 결정계수 계산
            adjusted_r_squared.append(adj_r_squared)  # 수정된 결정계수 저장
            sv_per_step.append(selected_variables.copy())  # 선택된 변수 리스트 저장
        else:
            break

    # 시각화
    fig = plt.figure(figsize=(100, 10))  # 그래프 크기 설정
    fig.set_facecolor('white')  # 배경색 설정
    
    font_size = 15
    plt.xticks(steps, [f'step {s}\n' + '\n'.join(sv_per_step[i]) for i, s in enumerate(steps)], fontsize=12)  # 단계별 선택된 변수 표시
    plt.plot(steps, adjusted_r_squared, marker='o')  # 수정된 결정계수 변화 시각화
      
    plt.ylabel('Adjusted R Squared', fontsize=font_size)  # y축 레이블 설정
    plt.grid(True)  # 격자 표시
    plt.show()  # 그래프 출력

    return selected_variables

# 단계적 선택법 실행
selected_variables = stepwise_feature_selection(X_train, y_train)


In [ ]:
#stepwise 적용 모델
stepwise_model = sm.OLS(y_train, sm.add_constant(pd.DataFrame(X_train[selected_variables]))).fit(disp=0)
stepwise_model.summary()


***

## **성능 비교(RMSE)**

In [ ]:
#full featrue
y_pred = model.predict(X_test)

rmse = sqrt(root_mean_squared_error(y_test, y_pred))
print("full_rmse : ",rmse)

#forward featrue
y_pred = forward_model.predict(X_test[forward_variables])

rmse = sqrt(root_mean_squared_error(y_test, y_pred))
print("forward_rmse : ",rmse)

#backward featrue
y_pred = back_model.predict(X_test[backward_variables_function])

rmse = sqrt(root_mean_squared_error(y_test, y_pred))
print("back_rmse : ",rmse)

#stepwise featrue
y_pred = stepwise_model.predict(X_test[selected_variables])

rmse = sqrt(root_mean_squared_error(y_test, y_pred))
print("stepwise_rmse : ",rmse)


***
## **5. 유전 알고리즘**

PCOS_data.csv
- 여성 호르몬 장애 환자들에 대한 데이터 셋
- 호르몬 이상을 경험한 환자들의 체내 특성 정보 포함

In [21]:
# 범주형 변수 컬럼 확인 및 라벨인코딩 함수
def preprocess_data(data):
    # 범주형 변수 확인
    categorical_columns = data.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        data[col] = LabelEncoder().fit_transform(data[col])
    return data

In [ ]:
# 데이터 로드
data = pd.read_csv("../data/PCOS_data.csv")
data = preprocess_data(data) # 라벨 인코딩
data.head()

In [23]:
# 데이터 전처리
# 'PCOS (Y/N)'을 종속 변수로 설정
X = data.drop(columns=['PCOS (Y/N)', 'Sl. No', 'Patient File No.', 'Unnamed: 44'], errors='ignore')  # 독립 변수
y = data['PCOS (Y/N)']


# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X.info()

In [ ]:
# 유전 알고리즘 설정
# Fitness 함수 정의 (정확도를 최대화하는 문제)
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

In [27]:
# 유전자의 표현 (특성 선택 여부를 0 또는 1로 표시)
toolbox.register("attr_bool", np.random.randint, 2)

# 개체 생성 (특성의 수만큼 0 또는 1로 이루어진 리스트)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=X.shape[1])

# 개체군 생성
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [28]:
def evaluate(individual):
    # 선택된 특성만으로 모델 학습 및 평가
    selected_features = [index for index, value in enumerate(individual) if value == 1]  # 선택된 특성의 인덱스 추출
    if len(selected_features) == 0:  # 특성을 하나도 선택하지 않은 경우 패널티 부여
        return 0.0,
    
    # 선택된 특성으로 학습 데이터와 테스트 데이터 구성
    X_train_selected = X_train.iloc[:, selected_features]  # 선택된 특성의 학습 데이터
    X_test_selected = X_test.iloc[:, selected_features]  # 선택된 특성의 테스트 데이터

    # 모델 학습 및 예측
    model = RandomForestClassifier(random_state=42)  # 랜덤 포레스트 모델 초기화
    model.fit(X_train_selected, y_train)  # 모델 학습
    predictions = model.predict(X_test_selected)  # 테스트 데이터 예측

    # 정확도 계산
    accuracy = accuracy_score(y_test, predictions)  # 예측 정확도 계산
    return accuracy,

# 교배, 변이, 선택 연산 정의
# GA(유전 알고리즘)에서 사용하는 연산자 등록
toolbox.register("evaluate", evaluate)  # 평가 함수 등록
toolbox.register("mate", tools.cxTwoPoint)  # 두 점 교배 연산 등록
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # 비트 플립 변이 연산 등록 (indpb는 변이 확률)
toolbox.register("select", tools.selTournament, tournsize=3)  # 토너먼트 선택 연산 등록 (tournsize는 토너먼트 크기)


In [ ]:
initial_model = RandomForestClassifier(random_state=42)
initial_model.fit(X_train, y_train)
initial_predictions = initial_model.predict(X_test)
initial_accuracy = accuracy_score(y_test, initial_predictions)
print(f"Initial accuracy (all features): {initial_accuracy:.4f}")

In [ ]:
# 유전 알고리즘으로 변수 선택 수행
population = toolbox.population(n=50)  # 초기 개체군 크기
ngen = 20  # 세대 수
cxpb = 0.5  # 교배 확률
mutpb = 0.2  # 변이 확률

# 통계 정보 출력 설정
stats = tools.Statistics(lambda ind: ind.fitness.values)  # 개체의 적합도 값에 대한 통계 설정
stats.register("avg", np.mean)  # 세대별 평균 적합도 계산
stats.register("min", np.min)  # 세대별 최소 적합도 계산
stats.register("max", np.max)  # 세대별 최대 적합도 계산

# 유전 알고리즘 실행
population, logbook = algorithms.eaSimple(
    population, toolbox, cxpb, mutpb, ngen, stats=stats, verbose=True  # 알고리즘 실행 및 통계 출력
)

# 최적의 해 찾기
best_individual = tools.selBest(population, k=1)[0]  # 최적의 개체 선택
selected_features = [index for index, value in enumerate(best_individual) if value == 1]  # 선택된 특성 인덱스 추출
print(f"Best individual: {best_individual}")  # 최적의 개체 출력
print(f"Selected features: {selected_features}")  # 선택된 특성 출력


In [ ]:
# 선택된 특성을 사용한 모델 정확도 계산
if len(selected_features) > 0:
    X_train_selected = X_train.iloc[:, selected_features]
    X_test_selected = X_test.iloc[:, selected_features]

    final_model = RandomForestClassifier(random_state=42)
    final_model.fit(X_train_selected, y_train)
    final_predictions = final_model.predict(X_test_selected)
    final_accuracy = accuracy_score(y_test, final_predictions)
    print(f"Final accuracy (selected features): {final_accuracy:.4f}")
else:
    final_accuracy = 0.0
    print("No features were selected.")

In [ ]:
improvement = final_accuracy - initial_accuracy
print(f"Accuracy improvement: {improvement:.4f}")